<a href="https://colab.research.google.com/github/Beerschtein/Portfolio/blob/main/Course_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Course project


**Задача: необходимо на тестовом датасете превысить показатель целевой метрики precision@5.**

**Целевая метрика precision@5 > 0.235**

*1. Бейзлайн решения - MainRecommender*

*2. Сдаем ссылку на github с решением.*

*3. В решении должны быть отчетливо видна метрика на новом тестовом сете из файла retail_test1.csv, то есть вам нужно для всех юзеров из этого файла выдать выши рекомендации, и посчитать на actual покупках precision@5.*

**Установим необходимые библиотеки и зрузим данные.**

In [1]:
!pip install implicit==0.4.4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 18.2 MB/s 
  Created wheel for implicit: filename=implicit-0.4.4-cp37-cp37m-linux_x86_64.whl size=3423199 sha256=4e835af08b3ae3afede9f2aa55ae7edc9b9055bb7c6e1bef367242b3011a9b8a
  Stored in directory: /root/.cache/pip/wheels/44/7e/7d/a17324ea207cfbe76aca878b5b8ca0aa932cf55d163329be37
Successfully built implicit


# Загружаем данные

**Для удобства и быстроты все необходимые файлы были заархивированы и размещены на GoogleDrive.**

In [2]:
!wget 'https://drive.google.com/uc?export=download&confirm=no_antivirus&id=1668qzODOw59CQltoFUg9IikgAC4lPdSK' -O course_project_data.zip

--2022-05-30 12:50:03--  https://drive.google.com/uc?export=download&confirm=no_antivirus&id=1668qzODOw59CQltoFUg9IikgAC4lPdSK
Resolving drive.google.com (drive.google.com)... 172.217.15.110, 2607:f8b0:4004:c17::8a, 2607:f8b0:4004:c17::8b, ...
Connecting to drive.google.com (drive.google.com)|172.217.15.110|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0g-18-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/hptms4pcvjc4oq4kocfpgd0bhh70lnpa/1653915000000/06840004559052771215/*/1668qzODOw59CQltoFUg9IikgAC4lPdSK?e=download [following]
--2022-05-30 12:50:04--  https://doc-0g-18-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/hptms4pcvjc4oq4kocfpgd0bhh70lnpa/1653915000000/06840004559052771215/*/1668qzODOw59CQltoFUg9IikgAC4lPdSK?e=download
Resolving doc-0g-18-docs.googleusercontent.com (doc-0g-18-docs.googleusercontent.com)... 142.250.188.193, 2607:f8b0:4004:836::2001
Connecting to doc-0g-18-do

In [3]:
!unzip course_project_data.zip

Archive:  course_project_data.zip
  inflating: recommenders.py         
  inflating: retail_test1.csv        
  inflating: retail_train.csv        
  inflating: transaction_data.csv    
  inflating: utils.py                
  inflating: hh_demographic.csv      
  inflating: metrics.py              
  inflating: product.csv             


# Import libs

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from lightgbm import LGBMClassifier

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные нами функции
from metrics import precision_at_k, recall_at_k
from utils import prefilter_items
from recommenders import MainRecommender

# Read data

In [5]:
data = pd.read_csv('./retail_train.csv')
item_features = pd.read_csv('./product.csv')
user_features = pd.read_csv('./hh_demographic.csv')

In [6]:
data.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2396804 entries, 0 to 2396803
Data columns (total 12 columns):
 #   Column             Dtype  
---  ------             -----  
 0   user_id            int64  
 1   basket_id          int64  
 2   day                int64  
 3   item_id            int64  
 4   quantity           int64  
 5   sales_value        float64
 6   store_id           int64  
 7   retail_disc        float64
 8   trans_time         int64  
 9   week_no            int64  
 10  coupon_disc        float64
 11  coupon_match_disc  float64
dtypes: float64(4), int64(8)
memory usage: 219.4 MB


In [8]:
item_features.head()

,PRODUCT_ID,MANUFACTURER,DEPARTMENT,BRAND,COMMODITY_DESC,SUB_COMMODITY_DESC,CURR_SIZE_OF_PRODUCT
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,
2,26093,69,PASTRY,Private,BREAD,BREAD:ITALIAN/FRENCH,
3,26190,69,GROCERY,Private,FRUIT - SHELF STABLE,APPLE SAUCE,50 OZ
4,26355,69,GROCERY,Private,COOKIES/CONES,SPECIALTY COOKIES,14 OZ


In [9]:
item_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92353 entries, 0 to 92352
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   PRODUCT_ID            92353 non-null  int64 
 1   MANUFACTURER          92353 non-null  int64 
 2   DEPARTMENT            92353 non-null  object
 3   BRAND                 92353 non-null  object
 4   COMMODITY_DESC        92353 non-null  object
 5   SUB_COMMODITY_DESC    92353 non-null  object
 6   CURR_SIZE_OF_PRODUCT  92353 non-null  object
dtypes: int64(2), object(5)
memory usage: 4.9+ MB


In [10]:
user_features.head()

,AGE_DESC,MARITAL_STATUS_CODE,INCOME_DESC,HOMEOWNER_DESC,HH_COMP_DESC,HOUSEHOLD_SIZE_DESC,KID_CATEGORY_DESC,household_key
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7
2,25-34,U,25-34K,Unknown,2 Adults Kids,3,1,8
3,25-34,U,75-99K,Homeowner,2 Adults Kids,4,2,13
4,45-54,B,50-74K,Homeowner,Single Female,1,None/Unknown,16


In [11]:
user_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 801 entries, 0 to 800
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   AGE_DESC             801 non-null    object
 1   MARITAL_STATUS_CODE  801 non-null    object
 2   INCOME_DESC          801 non-null    object
 3   HOMEOWNER_DESC       801 non-null    object
 4   HH_COMP_DESC         801 non-null    object
 5   HOUSEHOLD_SIZE_DESC  801 non-null    object
 6   KID_CATEGORY_DESC    801 non-null    object
 7   household_key        801 non-null    int64 
dtypes: int64(1), object(7)
memory usage: 50.2+ KB


In [12]:
data['week_no'].nunique()

95

In [13]:
users, items, interactions = data.user_id.nunique(), data.item_id.nunique(), data.shape[0]

print(f'# users: {users}')
print(f'# items: {items}')
print(f'# interactions: {interactions}')

# users: 2499
# items: 89051
# interactions: 2396804


# Set global const

In [14]:
ITEM_COL = 'item_id'
USER_COL = 'user_id'
ACTUAL_COL = 'actual'

# N = Neighbors
# N_PREDICT = 50 
# N_PREDICT = 200 
# N_PREDICT = 300 
N_PREDICT = 500 

# Process features dataset

In [15]:
# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': ITEM_COL}, inplace=True)
user_features.rename(columns={'household_key': USER_COL }, inplace=True)

# Split dataset for train, eval, test

In [16]:
# Важна схема обучения и валидации!
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)

VAL_MATCHER_WEEKS = 6
VAL_RANKER_WEEKS = 3

In [17]:
# берем данные для тренировки matching модели
data_train_matcher = data[data['week_no'] < data['week_no'].max() - (VAL_MATCHER_WEEKS + VAL_RANKER_WEEKS)]

# берем данные для валидации matching модели
data_val_matcher = data[(data['week_no'] >= data['week_no'].max() - (VAL_MATCHER_WEEKS + VAL_RANKER_WEEKS)) &
                      (data['week_no'] < data['week_no'].max() - (VAL_RANKER_WEEKS))]

# берем данные для тренировки ranking модели
data_train_ranker = data_val_matcher.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться

# берем данные для теста ranking, matching модели
data_val_ranker = data[data['week_no'] >= data['week_no'].max() - VAL_RANKER_WEEKS]

In [18]:
# сделаем объединенный сет данных для первого уровня (матчинга)
df_join_train_matcher = pd.concat([data_train_matcher, data_val_matcher])

In [19]:
def print_stats_data(df_data, name_df):
    print(name_df)
    print(f"Shape: {df_data.shape} Users: {df_data[USER_COL].nunique()} Items: {df_data[ITEM_COL].nunique()}")

In [20]:
print_stats_data(data_train_matcher,'train_matcher')
print_stats_data(data_val_matcher,'val_matcher')
print_stats_data(data_train_ranker,'train_ranker')
print_stats_data(data_val_ranker,'val_ranker')

train_matcher
Shape: (2108779, 12) Users: 2498 Items: 83685
val_matcher
Shape: (169711, 12) Users: 2154 Items: 27649
train_ranker
Shape: (169711, 12) Users: 2154 Items: 27649
val_ranker
Shape: (118314, 12) Users: 2042 Items: 24329


*Выше видим разброс по пользователям и товарам и дальше мы перейдем к warm-start (только известные пользователи).*

In [21]:
data_val_matcher.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
2104867,2070,40618492260,594,1019940,1,1.00,311,-0.29,40,86,0.0,0.0
2107468,2021,40618753059,594,840361,1,0.99,443,0.00,101,86,0.0,0.0


# Prefilter items

In [22]:
n_items_before = data_train_matcher['item_id'].nunique()

data_train_matcher = prefilter_items(data_train_matcher, item_features=item_features, take_n_popular=5000)

n_items_after = data_train_matcher['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

/content/utils.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['price'] = data['sales_value'] / (np.maximum(data['quantity'], 1))


Decreased # items from 83685 to 5001


# Make cold-start to warm-start

In [23]:
# ищем общих пользователей
common_users = list(set(data_train_matcher.user_id.values)&(set(data_val_matcher.user_id.values))&set(data_val_ranker.user_id.values))

# оставляем общих пользователей
data_train_matcher = data_train_matcher[data_train_matcher.user_id.isin(common_users)]
data_val_matcher = data_val_matcher[data_val_matcher.user_id.isin(common_users)]
data_train_ranker = data_train_ranker[data_train_ranker.user_id.isin(common_users)]
data_val_ranker = data_val_ranker[data_val_ranker.user_id.isin(common_users)]

print_stats_data(data_train_matcher,'train_matcher')
print_stats_data(data_val_matcher,'val_matcher')
print_stats_data(data_train_ranker,'train_ranker')
print_stats_data(data_val_ranker,'val_ranker')

train_matcher
Shape: (784420, 13) Users: 1915 Items: 4999
val_matcher
Shape: (163261, 12) Users: 1915 Items: 27118
train_ranker
Shape: (163261, 12) Users: 1915 Items: 27118
val_ranker
Shape: (115989, 12) Users: 1915 Items: 24042


# Init/train recommender

In [24]:
%%time
recommender = MainRecommender(data_train_matcher)

GPU training requires factor size to be a multiple of 32. Increasing factors from 20 to 32.


  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/4999 [00:00<?, ?it/s]

CPU times: user 4.27 s, sys: 454 ms, total: 4.72 s
Wall time: 7.52 s


**Варианты, как получить кандидатов**

*Можно потом все эти варианты соединить в один*

*(!) Если модель рекомендует < N товаров, то рекомендации дополняются топ-популярными товарами до N*

In [25]:
# Берем тестового юзера 2375

In [26]:
recommender.get_als_recommendations(2375, N=5)

[899624, 1106523, 871756, 1044078, 883932]

In [27]:
recommender.get_own_recommendations(2375, N=5)

[948640, 918046, 847962, 907099, 873980]

In [28]:
recommender.get_similar_items_recommendation(2375, N=5)

[1046545, 1044078, 1044078, 1072519, 15778319]

In [29]:
recommender.get_similar_users_recommendation(2375, N=5)

[970861, 1074040, 8019649, 873324, 1134296]

# Eval recall of matching

**Измеряем recall@k**

In [30]:
result_eval_matcher = data_val_matcher.groupby(USER_COL)[ITEM_COL].unique().reset_index()
result_eval_matcher.columns=[USER_COL, ACTUAL_COL]
result_eval_matcher.head(2)

,user_id,actual
0,1,"[853529, 865456, 867607, 872137, 874905, 87524..."
1,6,"[1024306, 1102949, 6548453, 835394, 940804, 96..."


In [31]:
%%time
# для понятности расписано все в строчку, без функций, ваша задача уметь оборачивать все это в функции
result_eval_matcher['own_rec'] = result_eval_matcher[USER_COL].apply(lambda x: recommender.get_own_recommendations(x, N=N_PREDICT))
result_eval_matcher['sim_item_rec'] = result_eval_matcher[USER_COL].apply(lambda x: recommender.get_similar_items_recommendation(x, N=N_PREDICT))
result_eval_matcher['als_rec'] = result_eval_matcher[USER_COL].apply(lambda x: recommender.get_als_recommendations(x, N=N_PREDICT))

CPU times: user 1min 5s, sys: 49.3 s, total: 1min 54s
Wall time: 1min 9s


In [32]:
%%time
# result_eval_matcher['sim_user_rec'] = result_eval_matcher[USER_COL].apply(lambda x: recommender.get_similar_users_recommendation(x, N=N_PREDICT))

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 6.91 µs


**Пример оборачивания**

In [33]:
# # сырой и простой пример как можно обернуть в функцию
def evalRecall(df_result, target_col_name, recommend_model):
    result_col_name = 'result'
    df_result[result_col_name] = df_result[target_col_name].apply(lambda x: recommend_model(x, N=25))
    return df_result.apply(lambda row: recall_at_k(row[result_col_name], row[ACTUAL_COL], k=N_PREDICT), axis=1).mean()

In [34]:
evalRecall(result_eval_matcher, USER_COL, recommender.get_own_recommendations)

0.04190749809147714

In [35]:
def calc_recall(df_data, top_k):
    for col_name in df_data.columns[2:]:
        yield col_name, df_data.apply(lambda row: recall_at_k(row[col_name], row[ACTUAL_COL], k=top_k), axis=1).mean()

In [36]:
def calc_precision(df_data, top_k):
    for col_name in df_data.columns[2:]:
        yield col_name, df_data.apply(lambda row: precision_at_k(row[col_name], row[ACTUAL_COL], k=top_k), axis=1).mean()

**Recall@50 of matching**

In [37]:
TOPK_RECALL = 50

In [38]:
sorted(calc_recall(result_eval_matcher, TOPK_RECALL), key=lambda x: x[1],reverse=True)

[('own_rec', 0.061684201353290766),
 ('als_rec', 0.05286296752264206),
 ('result', 0.04190749809147714),
 ('sim_item_rec', 0.03652229033641467)]

**Precision@5 of matching**

In [39]:
TOPK_PRECISION = 5

In [40]:
sorted(calc_precision(result_eval_matcher, TOPK_PRECISION), key=lambda x: x[1],reverse=True)

[('own_rec', 0.18872062663185182),
 ('result', 0.18872062663185182),
 ('als_rec', 0.1368146214099205),
 ('sim_item_rec', 0.07644908616188012)]

# Ranking part

**Обучаем модель 2-ого уровня на выбранных кандидатах**

*Обучаем на data_train_ranking*

*Обучаем только на выбранных кандидатах*

*Если юзер купил < "N_PREDICT" товаров, то "get_own_recommendations" дополнит рекоммендации топ-популярными*

In [41]:
# -- давние покупки -- | -- 6 недель -- | -- 3 недели -- 

# Подготовка данных для трейна

In [42]:
# взяли пользователей из трейна для ранжирования
df_match_candidates = pd.DataFrame(data_train_ranker[USER_COL].unique())
df_match_candidates.columns = [USER_COL]

In [43]:
# собираем кандитатов с первого этапа (matcher)
df_match_candidates['candidates'] = df_match_candidates[USER_COL].apply(lambda x: recommender.get_own_recommendations(x, N=N_PREDICT))

In [44]:
df_match_candidates.head(2)

,user_id,candidates
0,2070,"[1105426, 1097350, 879194, 948640, 928263, 944..."
1,2021,"[950935, 1119454, 835578, 863762, 1097398, 101..."


In [45]:
# разворачиваем товары
df_items = df_match_candidates.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
df_items.name = 'item_id'

In [46]:
df_match_candidates = df_match_candidates.drop('candidates', axis=1).join(df_items)

In [47]:
df_match_candidates.head(4)

,user_id,item_id
0,2070,1105426
0,2070,1097350
0,2070,879194
0,2070,948640


**Check warm start**

In [48]:
print_stats_data(df_match_candidates, 'match_candidates')

match_candidates
Shape: (957500, 2) Users: 1915 Items: 4504


**Создаем трейн сет для ранжирования с учетом кандидатов с этапа 1**

In [49]:
df_ranker_train = data_train_ranker[[USER_COL, ITEM_COL]].copy()
df_ranker_train['target'] = 1  # тут только покупки 

df_ranker_train.head()


,user_id,item_id,target
2104867,2070,1019940,1
2107468,2021,840361,1
2107469,2021,856060,1
2107470,2021,869344,1
2107471,2021,896862,1


In [50]:
df_ranker_train = df_match_candidates.merge(df_ranker_train, on=[USER_COL, ITEM_COL], how='left')

# чистим дубликаты
df_ranker_train = df_ranker_train.drop_duplicates(subset=[USER_COL, ITEM_COL])

df_ranker_train['target'].fillna(0, inplace= True)

In [51]:
df_ranker_train.target.value_counts()

0.0    878448
1.0     24947
Name: target, dtype: int64

In [52]:
df_ranker_train.head(2)

,user_id,item_id,target
0,2070,1105426,0.0
1,2070,1097350,0.0


In [53]:
df_ranker_train['target'].mean()

0.02761472002833755

# Подготавливаем фичи для обучения модели

**Описательные фичи**

In [54]:
item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [55]:
user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7


In [56]:
df_ranker_train = df_ranker_train.merge(item_features, on='item_id', how='left')
df_ranker_train = df_ranker_train.merge(user_features, on='user_id', how='left')

df_ranker_train.head(2)

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
0,2070,1105426,0.0,69,DELI,Private,SANDWICHES,SANDWICHES - (COLD),,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown
1,2070,1097350,0.0,2468,GROCERY,National,DOMESTIC WINE,VALUE GLASS WINE,4 LTR,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown


In [57]:
df_ranker_train.shape

(903395, 16)

*Фичи user_id: - Средний чек - Средняя сумма покупки 1 товара в каждой категории - Кол-во покупок в каждой категории - Частотность покупок раз/месяц - Долю покупок в выходные - Долю покупок утром/днем/вечером*

*Фичи item_id: - Кол-во покупок в неделю - Среднее ол-во покупок 1 товара в категории в неделю - (Кол-во покупок в неделю) / (Среднее ол-во покупок 1 товара в категории в неделю) - Цена (Можно посчитать из retil_train.csv) - Цена / Средняя цена товара в категории*

*Фичи пары user_id - item_id - (Средняя сумма покупки 1 товара в каждой категории (берем категорию item_id)) - (Цена item_id) - (Кол-во покупок юзером конкретной категории в неделю) - (Среднее кол-во покупок всеми юзерами конкретной категории в неделю) - (Кол-во покупок юзером конкретной категории в неделю) / (Среднее кол-во покупок всеми юзерами конкретной категории в неделю)*

**Поведенческие фичи**

*Чтобы считать поведенческие фичи, нужно учесть все данные что были до data_val_ranker*

In [58]:
df_join_train_matcher.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


**Создадим новые фичи, чтобы модель смогла отыскать дополнительные зависимости и закономерности.**

In [59]:
df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg('sales_value').sum().rename('total_item_sales_value'), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg('quantity').sum().rename('total_quantity_value'), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg(USER_COL).count().rename('item_freq'), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg(USER_COL).count().rename('user_freq'), how='left',on=USER_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg('sales_value').sum().rename('total_user_sales_value'), how='left',on=USER_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg('quantity').sum().rename('item_quantity_per_week')/df_join_train_matcher.week_no.nunique(), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg('quantity').sum().rename('user_quantity_per_week')/df_join_train_matcher.week_no.nunique(), how='left',on=USER_COL)


df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg('quantity').sum().rename('item_quantity_per_basket')/df_join_train_matcher.basket_id.nunique(), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg('quantity').sum().rename('user_quantity_per_baskter')/df_join_train_matcher.basket_id.nunique(), how='left',on=USER_COL)


df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg(USER_COL).count().rename('item_freq_per_basket')/df_join_train_matcher.basket_id.nunique(), how='left',on=ITEM_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg(USER_COL).count().rename('user_freq_per_basket')/df_join_train_matcher.basket_id.nunique(), how='left',on=USER_COL)

df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=[USER_COL,ITEM_COL]).agg('quantity').sum().rename('user_item_total_quantity_value'), how='left',on=[USER_COL,ITEM_COL])

# df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=[USER_COL,ITEM_COL]).agg('sales_value').sum().rename('total_user_item_sales_value'), how='left',on=[USER_COL,ITEM_COL])

# df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=[USER_COL,ITEM_COL]).agg('quantity').sum().rename('user_item_quantity_per_week')/df_join_train_matcher.week_no.nunique(), how='left',on=[USER_COL,ITEM_COL])

# df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=[USER_COL,ITEM_COL]).agg('quantity').sum().rename('user_item_quantity_per_basket')/df_join_train_matcher.basket_id.nunique(), how='left',on=[USER_COL,ITEM_COL])


In [60]:
df_ranker_train.tail()

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,...,item_freq,user_freq,total_user_sales_value,item_quantity_per_week,user_quantity_per_week,item_quantity_per_basket,user_quantity_per_baskter,item_freq_per_basket,user_freq_per_basket,user_item_total_quantity_value
903390,1745,948086,0.0,1646,PRODUCE,National,SALAD MIX,KITS,10 OZ,45-54,...,252,897,2397.25,3.329670,13.208791,0.001236,0.004902,0.001028,0.003658,NaN
903391,1745,12731714,0.0,3675,MEAT,National,SMOKED MEATS,HAM STEAKS/CUBES/SLICES,,45-54,...,258,897,2397.25,3.285714,13.208791,0.001219,0.004902,0.001052,0.003658,NaN
903392,1745,9707240,0.0,665,MEAT-PCKGD,National,HOT DOGS,PREMIUM - MEAT,14 OZ,45-54,...,293,897,2397.25,4.021978,13.208791,0.001493,0.004902,0.001195,0.003658,NaN
903393,1745,1121384,0.0,584,GROCERY,National,COLD CEREAL,ALL FAMILY CEREAL,21 OZ,45-54,...,286,897,2397.25,3.494505,13.208791,0.001297,0.004902,0.001166,0.003658,NaN
903394,1745,835530,0.0,1089,MEAT-PCKGD,National,HOT DOGS,PREMIUM - BEEF,16 OZ,45-54,...,370,897,2397.25,5.703297,13.208791,0.002117,0.004902,0.001509,0.003658,NaN


In [61]:
df_ranker_train.shape

(903395, 28)

In [62]:
X_train = df_ranker_train.drop('target', axis=1)
y_train = df_ranker_train[['target']]

In [63]:
cat_feats = X_train.columns[2:].tolist()
X_train[cat_feats] = X_train[cat_feats].astype('category')

cat_feats

['manufacturer',
 'department',
 'brand',
 'commodity_desc',
 'sub_commodity_desc',
 'curr_size_of_product',
 'age_desc',
 'marital_status_code',
 'income_desc',
 'homeowner_desc',
 'hh_comp_desc',
 'household_size_desc',
 'kid_category_desc',
 'total_item_sales_value',
 'total_quantity_value',
 'item_freq',
 'user_freq',
 'total_user_sales_value',
 'item_quantity_per_week',
 'user_quantity_per_week',
 'item_quantity_per_basket',
 'user_quantity_per_baskter',
 'item_freq_per_basket',
 'user_freq_per_basket',
 'user_item_total_quantity_value']

# Обучение модели ранжирования

In [64]:
%%time
lgb = LGBMClassifier(objective='binary',
                     max_depth=10,
                     n_estimators=1000,
                     learning_rate=0.05,
                    # DUSE_GPU=1,
                     categorical_column=cat_feats,
                    # early_stopping_round=20,
                     n_jobs=-1)

lgb.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


CPU times: user 3min 58s, sys: 337 ms, total: 3min 58s
Wall time: 2min 3s


In [65]:
%%time
train_preds = lgb.predict_proba(X_train)

CPU times: user 2min 42s, sys: 82.4 ms, total: 2min 42s
Wall time: 1min 22s


In [66]:
%%time
df_ranker_predict = df_ranker_train.copy()

CPU times: user 74.6 ms, sys: 167 µs, total: 74.8 ms
Wall time: 73.7 ms


In [67]:
%%time
df_ranker_predict['proba_item_purchase'] = train_preds[:,1]

CPU times: user 4.14 ms, sys: 0 ns, total: 4.14 ms
Wall time: 4.04 ms


**Подведем итоги**

*Мы обучили модель ранжирования на покупках из сета data_train_ranker и на кандитатах от own_recommendations, что является тренировочным сетом, и теперь наша задача предсказать и оценить именно на тестовом сете.*

# Evaluation on test dataset

In [68]:
result_eval_ranker = data_val_ranker.groupby(USER_COL)[ITEM_COL].unique().reset_index()
result_eval_ranker.columns=[USER_COL, ACTUAL_COL]
result_eval_ranker.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."


# Eval matching on test dataset

In [69]:
%%time
result_eval_ranker['own_rec'] = result_eval_ranker[USER_COL].apply(lambda x: recommender.get_own_recommendations(x, N=N_PREDICT))

CPU times: user 4.4 s, sys: 49.7 ms, total: 4.45 s
Wall time: 4.42 s


In [70]:
# померяем precision только модели матчинга, чтобы понимать влияение ранжирования на метрики

sorted(calc_precision(result_eval_ranker, TOPK_PRECISION), key=lambda x: x[1], reverse=True)

[('own_rec', 0.1462140992167092)]

# Eval re-ranked matched result on test dataset

*Вспомним df_match_candidates сет, который был получен own_recommendations на юзерах, набор пользователей мы фиксировали и он одинаков, значи и прогноз одинаков, поэтому мы можем использовать этот датафрейм для переранжирования.*

In [71]:
def rerank(user_id):
    return df_ranker_predict[df_ranker_predict[USER_COL]==user_id].sort_values('proba_item_purchase', ascending=False).head(5).item_id.tolist()

In [72]:
result_eval_ranker['reranked_own_rec'] = result_eval_ranker[USER_COL].apply(lambda user_id: rerank(user_id))

# Проверим данные метрики с фичами и без (есть ли какие-либо изменения):

In [73]:
print(*sorted(calc_precision(result_eval_ranker, TOPK_PRECISION), key=lambda x: x[1], reverse=True), sep='\n')

('reranked_own_rec', 0.31738903394255824)
('own_rec', 0.1462140992167092)


In [74]:
# смотрим на метрики выше и сравниваем что с ранжированием и без, добавляем фичи и то же смотрим

# Оценка на тесте для выполнения курсового проекта

In [75]:
df_test1 = pd.read_csv('./retail_test1.csv')
# df_transactions = pd.read_csv('./transaction_data.csv')

In [76]:
df_test1.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,1340,41652823310,664,912987,1,8.49,446,0.0,52,96,0.0,0.0
1,588,41652838477,664,1024426,1,6.29,388,0.0,8,96,0.0,0.0
2,2070,41652857291,664,995242,5,9.10,311,-0.6,46,96,0.0,0.0
3,1602,41665647035,664,827939,1,7.99,334,0.0,1741,96,0.0,0.0
4,1602,41665647035,664,927712,1,0.59,334,-0.4,1741,96,0.0,0.0


In [77]:
df_test1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88734 entries, 0 to 88733
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   user_id            88734 non-null  int64  
 1   basket_id          88734 non-null  int64  
 2   day                88734 non-null  int64  
 3   item_id            88734 non-null  int64  
 4   quantity           88734 non-null  int64  
 5   sales_value        88734 non-null  float64
 6   store_id           88734 non-null  int64  
 7   retail_disc        88734 non-null  float64
 8   trans_time         88734 non-null  int64  
 9   week_no            88734 non-null  int64  
 10  coupon_disc        88734 non-null  float64
 11  coupon_match_disc  88734 non-null  float64
dtypes: float64(4), int64(8)
memory usage: 8.1 MB


In [78]:
# df_transactions.head()

In [79]:
# df_transactions.info()

In [80]:
result_test1 = df_test1.groupby(USER_COL)[ITEM_COL].unique().reset_index()
result_test1.columns=[USER_COL, ACTUAL_COL]
result_test1.head(2)

,user_id,actual
0,1,"[880007, 883616, 931136, 938004, 940947, 94726..."
1,2,"[820165, 820291, 826784, 826835, 829009, 85784..."


In [81]:
#Чистим холодный старт, чтобы необученных юзеров не было в тестовом датасете
common_users = data_train_matcher.user_id.values

result_test1 = result_test1[result_test1.user_id.isin(common_users)]

In [82]:
#Строим базовые решения
result_test1['own_precision'] = result_test1[USER_COL].apply(lambda x: recommender.get_own_recommendations(x, N=N_PREDICT))

In [83]:
result_test1['reranked_own_precision'] = result_test1[USER_COL].apply(lambda user_id: rerank(user_id))

*Берем топ-k предсказаний, ранжированных по вероятности, для каждого юзера*

# Считаем precision@5 по новому тесту

In [84]:
print(*sorted(calc_precision(result_test1, TOPK_PRECISION), key=lambda x: x[1], reverse=True), sep='\n')

('reranked_own_precision', 0.2689116055321686)
('own_precision', 0.12928442573661988)


**ИТОГ: показатель 'reranked_own_precision' = 0.2689.**

При выполнении курсового проекта: исключил "холодный старт, построил матчинг, добавил фичи. В процессе тестирования модели часть фичей пришлось удалить (в том числе и парных), они отрицательно сказывались на результате. Увеличил значение N (neighbors) до 500 и сделал предварительную настройку LGBMClassifier.
Для улучшения финального результата можно попробовать сделать отбор фичей (уменьшить размерность), применить CatBoost и оптимизировать его параметры, попробовать синтезировать дополнительные фичи.